In [8]:
import pandas as pd
import sqlite3 as sqlite
from sqlite3 import Error
import sys
import os
import regex as re
import sqlalchemy
import tqdm

# import PySpark

# from pyspark import SparkConf, SparkContext

In [17]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [35]:
from __future__ import unicode_literals
import spacy,en_core_web_sm
from collections import Counter

In [2]:
db = "../data/yelp/yelp.db"
table_name = "Reviews"

In [11]:
# Create your connection.
cnx = sqlite.connect(db)

df = pd.read_sql_query("SELECT * FROM Reviews", cnx)

In [13]:
df.head()

,review_id,user_id,res_id,lat,lng,zipcode,review_year,review_date,text,useful,funny,cool,star
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,36.215546,-115.248798,89128,2013,2013-05-07 04:34:36,Total bill for this horrible service? Over $8G...,6,1,0,1.0
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,36.110083,-115.153871,89169,2017,2017-01-14 21:30:33,I *adore* Travis at the Hard Rock's new Kelly ...,0,0,0,5.0
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,33.259702,-111.790203,85286,2016,2016-11-09 20:09:03,I have to say that this office really has it t...,3,0,0,5.0
3,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,33.496713,-111.927958,85251,2018,2018-01-30 23:07:38,Today was my second out of three sessions I ha...,7,0,0,1.0
4,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,40.426779,-80.005652,15211,2013,2013-01-20 13:25:59,I'll be the first to admit that I was not exci...,0,0,0,4.0


In [14]:
trial1 = df.loc[0, "text"]

In [15]:
trial1

'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.'

In [18]:
nlp = spacy.load('en')

In [20]:
# for index, row in metadata_df.iterrows():
#     text = row["text"]
    
text = trial1
    
#first removing -- 
text = re.sub(r"-+"," ", text)

#remove non-alphabeticals
reg_text = re.sub(r'[^\w\s]', ' ', text)

#remove line breaks
reg_nolines = reg_text.replace(os.linesep, " ")

#remove tabs, and other whitespace chars
no_tabs = re.sub(r"\s+", " ", reg_nolines)

#remove numbers
no_nums = re.sub(r'[0-9]', '', no_tabs)

#remove stopwords
words_nostop = ""
for word in no_nums.split():
    if word not in STOP_WORDS:
        words_nostop+= " " + word

#     print(words_nostop)
#     metadata_df.set_value(index, 'text', words_nostop)

In [21]:
words_nostop

' Total bill horrible service Over Gs These crooks actually nerve charge pills I checked online pills cents EACH Avoid Hospital ERs costs'

In [22]:
doc = nlp(text)

In [81]:
for i in tqdm.tqdm(df.loc[:, "text"].head(10)):
    print(i)
    print("******")

100%|██████████| 10/10 [00:00<00:00, 6761.73it/s]

Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.
******
I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  

Travis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  Travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  That was superb!  Then starts the glorious blowout... where not one, not two, but THREE people were involved in doing the best round-brush action my hair 

In [ ]:
number_tokens

In [30]:
for token in doc:
#     print(token.text, token.pos_)
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Total total ADJ JJ amod Xxxxx True False
bill bill NOUN NN ROOT xxxx True False
for for ADP IN prep xxx True True
this this DET DT det xxxx True True
horrible horrible ADJ JJ amod xxxx True False
service service NOUN NN pobj xxxx True False
? ? PUNCT . punct ? False False
Over over ADP IN ROOT Xxxx True False
$ $ SYM $ nmod $ False False
8Gs 8gs NUM CD pobj dXx False False
. . PUNCT . punct . False False
These these DET DT det Xxxxx True False
crooks crook NOUN NNS nsubj xxxx True False
actually actually ADV RB advmod xxxx True False
had have VERB VBD ROOT xxx True True
the the DET DT det xxx True True
nerve nerve NOUN NN dobj xxxx True False
to to PART TO aux xx True True
charge charge VERB VB acl xxxx True False
us -PRON- PRON PRP dative xx True True
$ $ SYM $ nmod $ False False
69 69 NUM CD dobj dd False False
for for ADP IN prep xxx True True
3 3 NUM CD nummod d False False
pills pill NOUN NNS pobj xxxx True False
. . PUNCT . punct . False False
I -PRON- PRON PRP nsubj X True False

In [78]:
nlp = en_core_web_sm.load()

for i in tqdm(df.loc[:, "text"].head(10)):
    c = Counter(([token.pos_ for token in nlp(trial1)]))
    sbase = sum(c.values())
    pos_percent_dict = {}
    for el, cnt in c.items():
        pos_percent_dict[el] = '{0:2.4f}%'.format((cnt)/sbase)

TypeError: 'module' object is not callable

In [61]:
pos_percent_dict

{'ADJ': '0.0435%',
 'ADP': '0.1087%',
 'ADV': '0.0435%',
 'DET': '0.1304%',
 'NOUN': '0.2174%',
 'NUM': '0.0870%',
 'PART': '0.0217%',
 'PRON': '0.0435%',
 'PUNCT': '0.1087%',
 'SYM': '0.0435%',
 'VERB': '0.1522%'}

In [12]:
cols = [column[0] for column in df.description]

AttributeError: 'DataFrame' object has no attribute 'description'